In [ ]:
%pip install crewai  
%pip install langchain
# %pip install -q -U google-generativeai
# %pip install -qU langchain-google-genai

In [80]:
#from langchain_openai import ChatOpenAI
#from langchain_google_genai import ChatGoogleGenerativeAI

In [75]:
import warnings
warnings.filterwarnings('ignore')
from crewai import Agent, Task, Crew
import os
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool,BaseTool
from crewai import Process
import google.generativeai as genai


os.environ["OPENAI_API_KEY"] = "your key here"

# genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# my_llm = LLM(
#     provider="gemini",
#     api_key=os.getenv("GEMINI_API_KEY"),
#     model="gemini-pro",
#     temperature=0.5
# )


In [76]:
# Event Planning Agent
event_planner = Agent(
    role="Event Planner",
    goal="Plan engaging and student-friendly events for {event_type}",
    backstory="You're the creative mind behind planning exciting bakery events "
              "that resonate with college students. You understand student life, "
              "their schedules, and what makes an event appealing to them. "
              "You consider factors like exam periods, student budgets, "
              "and social media trends when planning events.",
    allow_delegation=True,
    verbose=True
)


# Message Creator Agent
message_creator = Agent(
    role="Message Creator",
    goal="Create trendy, engaging chat messages for {event_type} that appeal to college students",
    backstory="You're a social media-savvy chat moderator who stays current with "
              "Gen-Z trends and slang. You create messages that are authentic, "
              "relatable, and engaging while maintaining the bakery's warm brand voice. "
              "You use trending phrases, emojis, and current social media patterns "
              "to make the bakery's messages resonate with students.",
    allow_delegation=True,
    verbose=True
)

# Engagement Monitor Agent
engagement_monitor = Agent(
    role="Engagement Monitor",
    goal="Review and optimize chat messages for maximum student engagement",
    backstory="You're an expert in student engagement who ensures messages will "
              "generate positive responses and interaction. You review messages "
              "for timing, tone, and potential engagement factors. You suggest "
              "improvements based on previous successful interactions and "
              "current student preferences.",
    allow_delegation=True,
    verbose=True
)

translator = Agent(
    role="Translator",
    goal="Translate messages from English to Chinese",
    backstory="You're a translator who can translate messages from English to Chinese.",
    allow_delegation=True,
    verbose=True
)

# Tasks
plan_event = Task(
    description=(
        "1. Identify the type of event that would appeal to students "
        "based on {current_season} and {student_schedule}.\n"
        "2. Consider budget-friendly options and student preferences.\n"
        "3. Plan timing that works with typical student schedules.\n"
        "4. Include special offers or incentives that motivate participation.\n"
        "5. Consider ways to make the event shareable on social media."
    ),
    expected_output="A detailed event plan including type, timing, "
                    "special offers, and key selling points for students.",
    agent=event_planner
)

create_message = Task(
    description=(
        "1. Analyze current trends and popular phrases for the message type.\n"
        "2. Select appropriate trending phrases and emojis.\n"
        "3. Craft a message using the MessageStyler templates.\n"
        "4. Optimize message timing using EngagementOptimizer.\n"
        "5. Ensure the message maintains authenticity while being trendy.\n"
        "6. Include clear call-to-action with trending elements."
    ),
    expected_output="A trendy, engaging chat message incorporating current "
                    "Gen-Z language patterns and optimal posting time suggestions.",
    agent=message_creator
)

review_engagement = Task(
    description=(
        "1. Review the message for engagement potential.\n"
        "2. Check timing of the message.\n"
        "3. Ensure the tone matches student preferences.\n"
        "4. Verify all information is clear and actionable.\n"
        "5. Suggest improvements based on previous engagement data."
    ),
    expected_output="Reviewed and optimized message with "
                    "suggestions for maximum engagement.",
    agent=engagement_monitor
)


translate_message = Task(
    description=(
        "1. Translate the message from English to Chinese.\n"
    ),
    expected_output="A translated message in Chinese.",
    agent=translator
)


In [77]:
crew = Crew(
    agents=[event_planner, message_creator, engagement_monitor, translator],
    tasks=[plan_event, create_message, review_engagement, translate_message],
    tools=[SerperDevTool(), ScrapeWebsiteTool(), WebsiteSearchTool(), SentimenAnalysisTool()],
    #manager_llm = ChatOpenAI(temperature=0, model_name="gpt-4o"),
    #manager_llm = my_llm,
    #process=Process.hierarchical,
    verbose= True
)

2024-11-18 22:11:00,325 - 34240 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [78]:
event_details = {
    "event_type": "Finals Week Study Break",
    "current_season": "Winter",
    "student_schedule": "Finals Week"
}


In [79]:
result = crew.kickoff(inputs = event_details)

2024-11-18 22:11:02,871 - 34240 - _metadata.py-_metadata:142 - WARNING: Compute Engine Metadata server unavailable on attempt 1 of 3. Reason: [WinError 10051] 向一个无法连接的网络尝试了一个套接字操作。


# Agent: Crew Manager
## Task: 1. Identify the type of event that would appeal to students based on Winter and Finals Week.
2. Consider budget-friendly options and student preferences.
3. Plan timing that works with typical student schedules.
4. Include special offers or incentives that motivate participation.
5. Consider ways to make the event shareable on social media.


2024-11-18 22:11:03,948 - 34240 - _metadata.py-_metadata:142 - WARNING: Compute Engine Metadata server unavailable on attempt 2 of 3. Reason: [WinError 10051] 向一个无法连接的网络尝试了一个套接字操作。


KeyboardInterrupt: 

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)